<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Helium_Mobius_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# Helium (Hylleraas N<=2): S e V analíticos; T por Gauss–Laguerre
# H = T + V; resolve H c = E S c e imprime todos os autovalores
# ζ = 1.8, Z = 2.0 (ajuste "zeta_val" se quiser)
# ============================================================
import sympy as sp
import numpy as np
from mpmath import mp

# --------------------------
# Parâmetros globais
# --------------------------
zeta_val = 1.8
Z_val    = 2.0
mp.dps = 100  # mais robusto para N=3

# --------------------------
# Variáveis simbólicas
# --------------------------
u, v, w = sp.symbols('u v w', positive=True, real=True)
zeta, Z = sp.symbols('zeta Z', positive=True, real=True)

# Perimétricas e Jacobiano
r1  = (u+v)/2
r2  = (u+w)/2
r12 = (v+w)/2
J   = sp.Rational(1,8)*u*v*w*(u+v)*(u+w)*(v+w)

# --------------------------
# Base Hylleraas
# --------------------------
def hylleraas_basis(N, sector):
    B=[]
    if sector=="singlet":
        for i in range(N+1):
            for j in range(i+1):             # i >= j
                for k in range(N+1 - i - j):
                    B.append((i,j,k))
    else:
        for i in range(N+1):
            for j in range(i):               # i > j
                for k in range(N+1 - i - j):
                    B.append((i,j,k))
    return B

# φ e combinações (Φ_sym)
def phi(i,j,k):
    return (r1**i)*(r2**j)*(r12**k)*sp.exp(-zeta*(r1+r2))

def Phi_sym(i,j,k, sector):
    if sector=="singlet":
        return phi(i,j,k) if i==j else (phi(i,j,k)+phi(j,i,k))/sp.sqrt(2)
    else:
        return (phi(i,j,k)-phi(j,i,k))/sp.sqrt(2)

# Peso exponencial para integração (polinômio × EXPO)
EXPO = sp.exp(-2*zeta*u - zeta*v - zeta*w)

def integrate_poly_times_expo(poly_expr):
    poly_expanded = sp.expand(poly_expr)
    res = sp.Integer(0)
    for term in poly_expanded.as_ordered_terms():
        coeff = term.as_coeff_Mul()[0]
        mon   = term/coeff
        a = sp.degree(mon, u)
        b = sp.degree(mon, v)
        c = sp.degree(mon, w)
        res += coeff * sp.factorial(a)/(2*zeta)**(a+1) \
                     * sp.factorial(b)/(zeta)**(b+1) \
                     * sp.factorial(c)/(zeta)**(c+1)
    return sp.simplify(res)

def integrate_expr(expr):
    # expr deve ser (polinômio)*EXPO (sem denominadores)
    return integrate_poly_times_expo(sp.simplify(expr/EXPO))

# --------------------------
# Matrizes S e V (analíticas, sem denominadores)
# --------------------------
J_uv = sp.Rational(1,8)*u*v*w*(u+w)*(v+w)   # J/(u+v)
J_uw = sp.Rational(1,8)*u*v*w*(u+v)*(v+w)   # J/(u+w)
J_vw = sp.Rational(1,8)*u*v*w*(u+v)*(u+w)   # J/(v+w)

def overlap_matrix(basis, sector):
    n=len(basis)
    S=sp.zeros(n)
    for a,(i,j,k) in enumerate(basis):
        Pa = Phi_sym(i,j,k, sector)
        for b,(p,q,r) in enumerate(basis):
            if b<a: S[a,b]=S[b,a]; continue
            Pb = Phi_sym(p,q,r, sector)
            Sab = integrate_expr(Pa*Pb*J)
            S[a,b]=sp.simplify(Sab); S[b,a]=S[a,b]
    return S

def potential_matrix(basis, sector):
    n=len(basis)
    V=sp.zeros(n)
    for a,(i,j,k) in enumerate(basis):
        Pa = Phi_sym(i,j,k, sector)
        for b,(p,q,r) in enumerate(basis):
            if b<a: V[a,b]=V[b,a]; continue
            Pb = Phi_sym(p,q,r, sector)
            integrand = (-2*Z)*Pa*Pb*J_uv + (-2*Z)*Pa*Pb*J_uw + (2)*Pa*Pb*J_vw
            Vab = integrate_expr(integrand)
            V[a,b]=sp.simplify(Vab); V[b,a]=V[a,b]
    return V

# --------------------------
# T por quadratura Gauss–Laguerre (determinístico)
# --------------------------
def laguerre_rule(n):
    x, wts = np.polynomial.laguerre.laggauss(n)  # integra ∫ f(x)e^{-x} dx
    return [mp.mpf(xx) for xx in x], [mp.mpf(ww) for ww in wts]




x_nodes, x_wts = laguerre_rule(32)  # x = 2ζu
y_nodes, y_wts = laguerre_rule(32)  # y = ζv
z_nodes, z_wts = laguerre_rule(32)  # z = ζw


def make_Phi_lambda(i,j,k, sector):
    Pa = Phi_sym(i,j,k, sector).subs({zeta:zeta_val, Z:Z_val})
    return sp.lambdify((u,v,w), Pa, "mpmath")

def kinetic_matrix_GL(basis, sector):
    n = len(basis)
    Phi_funcs = [make_Phi_lambda(i,j,k, sector) for (i,j,k) in basis]
    T = mp.matrix(n, n)
    z = mp.mpf(zeta_val)
    eps = mp.mpf('1e-30')  # blindagem numérica

    for a,(i,j,k) in enumerate(basis):
        for b,(p,q,r_) in enumerate(basis):
            if b < a:
                T[a,b] = T[b,a]
                continue

            def integrand_uvw(u_, v_, w_):
                den_uv = (u_+v_) + eps
                den_uw = (u_+w_) + eps
                den_vw = (v_+w_) + eps

                ir1  = 2/den_uv
                ir2  = 2/den_uw
                ir12 = 2/den_vw

                cos1 = ((u_+v_)**2 + (v_+w_)**2 - (u_+w_)**2) / (2*den_uv*den_vw)
                cos2 = ((u_+w_)**2 + (v_+w_)**2 - (u_+v_)**2) / (2*den_uw*den_vw)

                A_a = i*ir1 - z
                C_a = j*ir2 - z
                B_a = k*ir12
                A_b = p*ir1 - z
                C_b = q*ir2 - z
                B_b = r_*ir12

                Pa = Phi_funcs[a](u_, v_, w_)
                Pb = Phi_funcs[b](u_, v_, w_)
                Jv = (u_*v_*w_)*(u_+v_)*(u_+w_)*(v_+w_)/8

                term = mp.mpf('0.5') * (
                    (A_a*A_b + B_a*B_b + (A_a*B_b + A_b*B_a)*cos1) +
                    (C_a*C_b + B_a*B_b - (C_a*B_b + C_b*B_a)*cos2)
                )
                return term * Pa * Pb * Jv

            # u = x/(2ζ), v = y/ζ, w = z/ζ ; du dv dw = dx dy dz / (2 ζ^3)
            acc = mp.mpf('0')
            for (ix, wx) in zip(x_nodes, x_wts):
                u_ = ix/(2*z)
                for (iy, wy) in zip(y_nodes, y_wts):
                    v_ = iy/z
                    for (iz, wz) in zip(z_nodes, z_wts):
                        w_ = iz/z
                        acc += wx * wy * wz * integrand_uvw(u_, v_, w_)
            Tval = acc / (2 * z**3)
            T[a,b] = Tval
            T[b,a] = Tval

    # para numpy
    return np.array([[float(T[i,j]) for j in range(n)] for i in range(n)], dtype=float)


# =========================
# Kato cusp constraints (singlet, N=2)
# =========================
# =========================
# Kato cusp constraints (singlet, N=2) — base simetrizada (i>=j)
# =========================
import numpy as np
import math

def cusp_constraints_matrix(basis, zeta_val, Z_val):
    """
    Impõe, na base singlet com i>=j:
      (1) c_(0,0,1) = 1/2 * c_(0,0,0)
      (2) c_(1,0,0)_sym = -sqrt(2)*(Z - zeta) * c_(0,0,0)
    Retorna matriz C tal que C @ c = 0.
    """
    idx = {t:a for a,t in enumerate(basis)}
    need = [(0,0,0), (0,0,1), (1,0,0)]  # (0,1,0) não existe na base sim; já está em (1,0,0)_sym
    if not all(t in idx for t in need):
        return None

    rows = []

    # (1) e–e cusp: c001 - 1/2 c000 = 0
    row1 = np.zeros(len(basis), dtype=float)
    row1[idx[(0,0,1)]] = 1.0
    row1[idx[(0,0,0)]] = -0.5
    rows.append(row1)

    # (2) e–n cusp (base sim): c_(1,0,0)_sym + sqrt(2)*(Z - zeta)*c000 = 0
    row2 = np.zeros(len(basis), dtype=float)
    row2[idx[(1,0,0)]] = 1.0
    row2[idx[(0,0,0)]] = math.sqrt(2.0)*(Z_val - zeta_val)
    rows.append(row2)

    return np.vstack(rows)


def project_with_constraints(S, H, C):
    U, svals, Vt = np.linalg.svd(C, full_matrices=True)
    V = Vt.T  # n x n
    # quantas restrições efetivas existem?
    rankC = np.sum(svals > 1e-12)
    # nullspace = últimas colunas de V
    Q = V[:, rankC:]
    Sp = Q.T @ S @ Q
    Hp = Q.T @ H @ Q
    return Sp, Hp, Q

def solve_generalized_constrained(S, H, C):
    if C is None:
        # sem restrições
        evals, evecs = np.linalg.eigh(S)
        keep = evals > 1e-14
        Sinv2 = evecs[:,keep] @ np.diag(1.0/np.sqrt(evals[keep])) @ evecs[:,keep].T
        Ht = Sinv2 @ H @ Sinv2
        E, _ = np.linalg.eigh(Ht)
        return np.sort(E)
    # com restrições
    Sp, Hp, Q = project_with_constraints(S, H, C)
    evals, evecs = np.linalg.eigh(Sp)
    keep = evals > 1e-14
    Sinv2 = evecs[:,keep] @ np.diag(1.0/np.sqrt(evals[keep])) @ evecs[:,keep].T
    Ht = Sinv2 @ Hp @ Sinv2
    E, _ = np.linalg.eigh(Ht)
    return np.sort(E)


# --------------------------
# Solver do generalizado
# --------------------------
def to_numpy(Msym):
    return np.array(Msym.subs({zeta:zeta_val, Z:Z_val}), dtype=float)

def solve_generalized(S_sym, H_mat):
    S = to_numpy(S_sym) if isinstance(S_sym, sp.Matrix) else S_sym
    H = to_numpy(H_mat)  if isinstance(H_mat,  sp.Matrix) else H_mat
    evals, evecs = np.linalg.eigh(S)
    keep = evals > 1e-14
    Sinv2 = evecs[:,keep] @ np.diag(1.0/np.sqrt(evals[keep])) @ evecs[:,keep].T
    Ht = Sinv2 @ H @ Sinv2
    E, _ = np.linalg.eigh(Ht)
    return np.sort(E)

# ==========================
# Execução (N=2)
# ==========================
N = 2
B_s = hylleraas_basis(N, "singlet")
B_t = hylleraas_basis(N, "triplet")

print("Bases:")
print("Singlet:", B_s)
print("Triplet:", B_t)

print("\nConstruindo S e V (analíticos)…")
S_s = overlap_matrix(B_s, "singlet")
V_s = potential_matrix(B_s, "singlet")
S_t = overlap_matrix(B_t, "triplet")
V_t = potential_matrix(B_t, "triplet")

print("Construindo T por Gauss–Laguerre (alguns segundos)…")
T_s = kinetic_matrix_GL(B_s, "singlet")
T_t = kinetic_matrix_GL(B_t, "triplet")

H_s = T_s + to_numpy(V_s)
H_t = T_t + to_numpy(V_t)

Es = solve_generalized(S_s, H_s)
Et = solve_generalized(S_t, H_t)

np.set_printoptions(precision=8, suppress=True)
print("\n=== AUTOVALORES (ζ=1.8, Z=2.0) ===")
print("Singlet (todos):", Es)
print("Triplet (todos):", Et)

print("\nFundamentais:")
print("E0(singlet) =", Es[0])
print("E0(triplet) =", Et[0])

# ==========================
# Execução (N=2) com cúspides Kato no singlet
# ==========================
N = 2
B_s = hylleraas_basis(N, "singlet")
B_t = hylleraas_basis(N, "triplet")

print("Bases:")
print("Singlet:", B_s)
print("Triplet:", B_t)

print("\nConstruindo S e V (analíticos)…")
S_s_sym = overlap_matrix(B_s, "singlet")
V_s_sym = potential_matrix(B_s, "singlet")
S_t_sym = overlap_matrix(B_t, "triplet")
V_t_sym = potential_matrix(B_t, "triplet")

print("Construindo T por Gauss–Laguerre…")
T_s = kinetic_matrix_GL(B_s, "singlet")
T_t = kinetic_matrix_GL(B_t, "triplet")

# converter S e V simbólicos para numpy
def to_numpy(Msym): return np.array(Msym.subs({zeta:zeta_val, Z:Z_val}), dtype=float)
S_s = to_numpy(S_s_sym); V_s = to_numpy(V_s_sym); H_s = T_s + V_s
S_t = to_numpy(S_t_sym); V_t = to_numpy(V_t_sym); H_t = T_t + V_t

# montar restrições de Kato para o singlet
C_s = cusp_constraints_matrix(B_s, zeta_val, Z_val)   # 3x7
print("\nCusp constraints (singlet) shape:", None if C_s is None else C_s.shape)

# resolver com/sem restrições
Es_free  = solve_generalized(S_s_sym, H_s)                      # sem Kato (referência)
Es_cusp  = solve_generalized_constrained(S_s, H_s, C_s)         # com Kato
Et       = solve_generalized(S_t_sym, H_t)                      # triplet (sem Kato aqui)

np.set_printoptions(precision=8, suppress=True)
print("\n=== AUTOVALORES (ζ=1.8, Z=2.0) ===")
print("Singlet (livre):", Es_free)
print("Singlet (Kato): ", Es_cusp)
print("Triplet:        ", Et)
print("\nFundamentais:")
print("E0(singlet livre) =", Es_free[0])
print("E0(singlet Kato)  =", Es_cusp[0])
print("E0(triplet)       =", Et[0])



# ============================================
# Varredura em ζ com Kato (N=2)  + opções N=3 e nó no triplet
# Reutiliza: hylleraas_basis, overlap_matrix, potential_matrix,
#            kinetic_matrix_GL, cusp_constraints_matrix,
#            project_with_constraints, solve_generalized_constrained
#            e to_numpy já definidos acima.
# ============================================

# --------- toggles ---------
DO_SCAN_N2  = True   # mantém o scan N=2 se quiser comparar
DO_N3       = True   # ativa N=3
TRIPLET_NODE = True  # impõe nó em r12=0 no triplet (remove k=0)

# Utilitário: resolver setorial com (ou sem) restrições
def solve_sector(basis, sector, zeta_val, Z_val, apply_kato_singlet=False):
    S_sym = overlap_matrix(basis, sector)
    V_sym = potential_matrix(basis, sector)
    T_num = kinetic_matrix_GL(basis, sector)

    S = to_numpy(S_sym)
    V = to_numpy(V_sym)
    H = T_num + V

    if apply_kato_singlet and sector=="singlet":
        C = cusp_constraints_matrix(basis, zeta_val, Z_val)
        E = solve_generalized_constrained(S, H, C)
    else:
        E = solve_generalized(S_sym, H)
    return E

# Construção da base com opção de excluir k=0 no triplet (nó em r12=0)
def hylleraas_basis_with_node(N, sector, triplet_node=False):
    B = hylleraas_basis(N, sector)
    if sector=="triplet" and triplet_node:
        B = [t for t in B if t[2] >= 1]  # força r12^k com k>=1
    return B

# --------- (1) Varredura em ζ com N=2 + Kato no singlet ---------
if DO_SCAN_N2:
    print("\n=== Scan in ζ (N=2) with Kato in Singlet ===")
    z_grid = np.linspace(1.4, 2.4, 11)
    best_s = (1e9, None); best_t=(1e9, None)
    for zeta_val in z_grid:
        # bases N=2
        B_s = hylleraas_basis_with_node(2, "singlet", triplet_node=False)
        B_t = hylleraas_basis_with_node(2, "triplet", triplet_node=TRIPLET_NODE)

        # singlet com Kato
        Es = solve_sector(B_s, "singlet", zeta_val, Z_val, apply_kato_singlet=True)
        # triplet: sem Kato (ou com nó via base)
        Et = solve_sector(B_t, "triplet", zeta_val, Z_val, apply_kato_singlet=False)

        E0s = float(Es[0]); E0t = float(Et[0])
        print(f"ζ={zeta_val:4.2f}  E0_singlet(Kato)={E0s: .6f}   E0_triplet={'(node) ' if TRIPLET_NODE else ''}{E0t: .6f}")
        if E0s < best_s[0]: best_s = (E0s, zeta_val)
        if E0t < best_t[0]: best_t = (E0t, zeta_val)

    print("\n>>> Best (N=2):")
    print(f"Singlet(Kato):  E0={best_s[0]: .6f} at ζ={best_s[1]:.3f}")
    print(f"Triplet{'(node)' if TRIPLET_NODE else ''}: E0={best_t[0]: .6f} at ζ={best_t[1]:.3f}")
    print("Benchmark (Pekeris singlet): -2.903724 a.u.\n")

# --------- (2) N=3 com Kato no singlet (ativar DO_N3=True) ---------
if DO_N3:
    print("\n=== N=3 with Kato in Singlet (single ζ line) ===")
    zeta_val = 1.8  # mude aqui ou varra como acima
    # dica: aumentar ordem da quadratura ajuda N=3:
    # (redefina x_nodes,y_nodes,z_nodes acima com laguerre_rule(28) ou (32))
    B_s3 = hylleraas_basis_with_node(3, "singlet", triplet_node=False)
    B_t3 = hylleraas_basis_with_node(3, "triplet", triplet_node=TRIPLET_NODE)

    Es3 = solve_sector(B_s3, "singlet", zeta_val, Z_val, apply_kato_singlet=True)
    Et3 = solve_sector(B_t3, "triplet", zeta_val, Z_val, apply_kato_singlet=False)

    print(f"ζ={zeta_val:.2f}  N=3  E0_singlet(Kato)={float(Es3[0]): .6f}   E0_triplet={'(node) ' if TRIPLET_NODE else ''}{float(Et3[0]): .6f}")
    print(f"Dim singlet={len(B_s3)}, triplet={'(node) ' if TRIPLET_NODE else ''}{len(B_t3)}")

print("\n=== N=3 with Kato (singlet) + triplet node — scan in ζ ===")
z_grid = np.linspace(1.6, 2.4, 5)   # pode alargar p/ 1.5–2.6
best_s3 = (1e9, None); best_t3 = (1e9, None)
for zeta_val in z_grid:
    B_s3 = hylleraas_basis_with_node(3, "singlet", triplet_node=False)
    B_t3 = hylleraas_basis_with_node(3, "triplet", triplet_node=TRIPLET_NODE)
    Es3 = solve_sector(B_s3, "singlet", zeta_val, Z_val, apply_kato_singlet=True)
    Et3 = solve_sector(B_t3, "triplet", zeta_val, Z_val, apply_kato_singlet=False)
    e0s, e0t = float(Es3[0]), float(Et3[0])
    print(f"ζ={zeta_val:4.2f}  N=3  E0_singlet(Kato)={e0s: .6f}   E0_triplet(node)={e0t: .6f}")
    if e0s < best_s3[0]: best_s3 = (e0s, zeta_val)
    if e0t < best_t3[0]: best_t3 = (e0t, zeta_val)

print(f"\n>>> Best N=3:  Singlet(Kato) E0={best_s3[0]: .6f} @ ζ={best_s3[1]:.3f}")
print(f"               Triplet(node) E0={best_t3[0]: .6f} @ ζ={best_t3[1]:.3f}")


# --------- (3) Nota sobre o triplet com nó ---------
if TRIPLET_NODE:
    print("\n[Triplet] Node at r12=0 enforced by removing k=0 terms in the triplet basis.")
    print("Isso aproxima a condição de anulamento na coalescência para like-spin.")


#################################

# ================== CONFIG N=4 / ζ-scan / GL-60 ==================
import numpy as np
import math
import mpmath as mp
mp.dps = 100  # precisão alta para quadraturas estáveis

# Se a sua função laguerre_rule(n) existir, aumente a ordem da quadratura:
# (Descomente a linha certa no seu código onde define x_nodes, y_nodes, z_nodes)
# Ex.: x_nodes, x_wts = laguerre_rule(60)
#      y_nodes, y_wts = laguerre_rule(60)
#      z_nodes, z_wts = laguerre_rule(60)

Z_val = 2.0
zetas = [2.0, 2.2, 2.4, 2.6]
N = 4

# ================== CÚSPIDES (singlet) ==================
def cusp_constraints_matrix(basis, zeta_val, Z_val):
    """
    Base singlet simetrizada (i>=j). Impõe:
      (1) c_(0,0,1) = 1/2 * c_(0,0,0)
      (2) c_(1,0,0)_sym = -sqrt(2)*(Z - zeta)*c_(0,0,0)
    Retorna C tal que C @ c = 0 (2 x dim).
    """
    idx = {t:a for a,t in enumerate(basis)}
    need = [(0,0,0), (0,0,1), (1,0,0)]
    if not all(t in idx for t in need):
        return None
    rows = []
    # (1) e–e
    r1 = np.zeros(len(basis)); r1[idx[(0,0,1)]] = 1.0; r1[idx[(0,0,0)]] = -0.5
    rows.append(r1)
    # (2) e–n (base sim)
    r2 = np.zeros(len(basis)); r2[idx[(1,0,0)]] = 1.0; r2[idx[(0,0,0)]] = math.sqrt(2.0)*(Z_val - zeta_val)
    rows.append(r2)
    return np.vstack(rows)

def project_with_constraints(S, H, C):
    U, svals, Vt = np.linalg.svd(C, full_matrices=True)
    V = Vt.T
    rankC = int(np.sum(svals > 1e-12))
    Q = V[:, rankC:]  # base do núcleo de C
    Sp = Q.T @ S @ Q
    Hp = Q.T @ H @ Q
    return Sp, Hp, Q

def solve_generalized_constrained(S, H, C):
    if C is None:
        # fallback sem restrições
        evals, evecs = np.linalg.eigh(S)
        keep = evals > 1e-14
        Sinv2 = evecs[:,keep] @ np.diag(1.0/np.sqrt(evals[keep])) @ evecs[:,keep].T
        Ht    = Sinv2 @ H @ Sinv2
        E, _  = np.linalg.eigh(Ht)
        return np.sort(E)
    Sp, Hp, Q = project_with_constraints(S, H, C)
    evals, evecs = np.linalg.eigh(Sp)
    keep = evals > 1e-14
    Sinv2 = evecs[:,keep] @ np.diag(1.0/np.sqrt(evals[keep])) @ evecs[:,keep].T
    Ht    = Sinv2 @ Hp @ Sinv2
    E, _  = np.linalg.eigh(Ht)
    return np.sort(E)

# ================== TRIPLET: nó em r12=0 (remover k=0) ==================
def hylleraas_basis_with_node(N, sector, triplet_node=True):
    B = hylleraas_basis(N, sector)
    if sector == "triplet" and triplet_node:
        B = [t for t in B if t[2] >= 1]
    return B

# ================== RESOLUÇÃO SETORIAL (usa suas funções S,V,T) ==================
def solve_sector(basis, sector, zeta_val, Z_val, apply_kato_singlet=False):
    # construir S,V (analíticos no seu código) e T (GL numérica)
    S_sym = overlap_matrix(basis, sector)
    V_sym = potential_matrix(basis, sector)
    T_num = kinetic_matrix_GL(basis, sector)  # certifique-se que usa GL(60)

    S = to_numpy(S_sym).astype(float)
    V = to_numpy(V_sym).astype(float)
    H = T_num + V

    if apply_kato_singlet and sector == "singlet":
        C = cusp_constraints_matrix(basis, zeta_val, Z_val)
        E = solve_generalized_constrained(S, H, C)
    else:
        # resolver sem restrições
        evals, evecs = np.linalg.eigh(S)
        keep = evals > 1e-14
        Sinv2 = evecs[:,keep] @ np.diag(1.0/np.sqrt(evals[keep])) @ evecs[:,keep].T
        Ht    = Sinv2 @ H @ Sinv2
        E, _  = np.linalg.eigh(Ht)
        E = np.sort(E)
    return E

# ================== DRIVER N=4 / ζ ∈ {2.0, 2.2, 2.4, 2.6} ==================
print(f"Bases (N={N}):")
B_s = hylleraas_basis_with_node(N, "singlet", triplet_node=False)
B_t = hylleraas_basis_with_node(N, "triplet", triplet_node=True)
print("Singlet:", B_s)
print("Triplet(node):", B_t)

print("\n=== Scan ζ (N=4), GL(60), Kato(singlet) & node(triplet) ===")
best_s = (1e9, None)
best_t = (1e9, None)

print(f"{'ζ':>4s}  {'E0_singlet(Kato)':>18s}  {'E0_triplet(node)':>18s}")
for zeta_val in zetas:
    Es = solve_sector(B_s, "singlet", zeta_val, Z_val, apply_kato_singlet=True)
    Et = solve_sector(B_t, "triplet", zeta_val, Z_val, apply_kato_singlet=False)
    e0s = float(Es[0]); e0t = float(Et[0])
    print(f"{zeta_val:4.1f}  {e0s:18.6f}  {e0t:18.6f}")
    if e0s < best_s[0]: best_s = (e0s, zeta_val)
    if e0t < best_t[0]: best_t = (e0t, zeta_val)

print("\n>>> Best (N=4):")
print(f"Singlet(Kato):   E0 = {best_s[0]:.6f}  @ ζ = {best_s[1]:.1f}")
print(f"Triplet(node):   E0 = {best_t[0]:.6f}  @ ζ = {best_t[1]:.1f}")
print("\nBenchmark (Pekeris singlet) = -2.903724 a.u.")


Bases:
Singlet: [(0, 0, 0), (0, 0, 1), (0, 0, 2), (1, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0)]
Triplet: [(1, 0, 0), (1, 0, 1), (2, 0, 0)]

Construindo S e V (analíticos)…
Construindo T por Gauss–Laguerre (alguns segundos)…

=== AUTOVALORES (ζ=1.8, Z=2.0) ===
Singlet (todos): [-1.7397474  -1.33348908 -1.24518583 -0.97340903 -0.85864954 -0.75548188
 -0.60478674]
Triplet (todos): [-1.95695964 -1.21233583 -0.82178515]

Fundamentais:
E0(singlet) = -1.7397474014187853
E0(triplet) = -1.95695964461063
Bases:
Singlet: [(0, 0, 0), (0, 0, 1), (0, 0, 2), (1, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0)]
Triplet: [(1, 0, 0), (1, 0, 1), (2, 0, 0)]

Construindo S e V (analíticos)…
Construindo T por Gauss–Laguerre…

Cusp constraints (singlet) shape: (2, 7)

=== AUTOVALORES (ζ=1.8, Z=2.0) ===
Singlet (livre): [-1.7397474  -1.33348908 -1.24518583 -0.97340903 -0.85864954 -0.75548188
 -0.60478674]
Singlet (Kato):  [-1.68535416 -1.09952653 -0.96646344 -0.80513111 -0.61245461]
Triplet:         [-1.95695964 -1.2123